In [1]:
# Original filename: "02_tessman_flow_reqts.py"

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
# Paths, input, and output file names

outer_dir = os.path.abspath(os.path.join(os.getcwd() ,"../.."))
refpth = os.path.join(outer_dir, 'IFT_files', 'Reference Files')
sfe_char_csv = os.path.join(refpth, 'TNC SFE LOI Characteristics with MAF.csv')
sfe_char_xls= os.path.join(refpth, 'TNC SFE LOI Characteristics.xlsx')
subset_xls = os.path.join(refpth, 'TNC Subset SFE LOIs.xlsx')
#sfe_char_csv = os.path.join(refpth, 'All SFE LOI Characteristics with MAF.csv')
#sfe_char_xls= os.path.join(refpth, 'All SFE LOI Characteristics.xlsx')
#subset_xls = os.path.join(refpth, 'Subset SFE LOIs.xlsx')
unimpath = os.path.join(outer_dir, "IFT_files", 'Unimpaired Flow')
#tesreqtsfile = os.path.join(refpth, 'Tessmann_min_flows_SWB.csv') #provided Tessmann table

startdir = os.path.join(outer_dir, 'IFT_files', 'IFT Results')
alldates = pd.date_range(dt.datetime(1950,10,1),dt.datetime(2021,9,30))
out_csv = os.path.join(startdir, 'All LOI Tessmann Flow Requirements_TNC.csv')

In [3]:
# Original filename: "00_get_all_sfe_lois.py"
# Read in SFE LOI characteristics table and calculate bankfull flow using cont. Area relation

def get_all_sfe_lois():
    sfelois = pd.read_csv(sfe_char_csv,index_col=0)
    loi = [str(i) for i in sfelois['LOI']]
    sfelois['Outlet LOI'] = sfelois['LOI']
    sfelois['Contributing Area (mi^2)'] = sfelois['Contributing Area']
    sfelois['MAF'] = sfelois['Mean Annual Flow (cfs)']
    sfelois['Qbf'] = 71.5 * sfelois['Contributing Area (mi^2)'] #71.5 cfs/mi^2 according to Darren

    subset = pd.read_excel(subset_xls)
    sublois = sfelois.loc[subset['SWSID'],:]
    return sublois, loi, sfelois

In [4]:
# Original filename: "00_read_loi_paradigm_flow_v3.py"
def read_loi_paradigm_flow(p):
    unimpflowfile = os.path.join(unimpath, str(p) + '.csv')
    unimp = pd.read_csv(unimpflowfile,index_col=0)
    unimp.index = pd.DatetimeIndex(unimp.index)
    return unimp

In [5]:
# Original filename: "00_add_ts_col.py"
def add_ts_col(tab):
#adds column for TS as first column of DataFrame as formatted date
    tab['TS'] = tab.index.strftime('%m/%d/%Y')
    cols = tab.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    tab = tab[cols]
    return tab

In [6]:
#tesreqts = pd.read_csv(tesreqtsfile)
loitab, loi, fultab = get_all_sfe_lois()

In [7]:
loitab

,LOI,dnLOI,Lat,Lon,Contributing Area,Geomorphic Class,Geologic Setting,Mean Annual Flow (cfs),COMID,Outlet LOI,Contributing Area (mi^2),MAF,Qbf
2664961,Indian,NaN,NaN,NaN,39.486536,NaN,NaN,63.202346,2664961,Indian,39.486536,63.202346,2823.287303
8285498,Sproul,NaN,NaN,NaN,4.992762,NaN,NaN,17.108696,8285498,Sproul,4.992762,17.108696,356.982496


In [8]:
dailyreqts = pd.DataFrame(index=alldates,columns=loi)  #set up IFT table for WEAP

In [9]:
min_days_month = 20
min_days_year = 360

for l in loi[:]:
    print(l)
    df = read_loi_paradigm_flow(l)
    df['datetime'] = df.index
    # Calculate mean monthly flow, screening out months with too much missing data
    df_month = (df.groupby(['Water Year', 'Month'])
           .agg({'Day':'count', 'flow':'mean'})
           .reset_index()
           .rename(columns={'Day':'count_days','flow':'monthly_mean_flow'})
          )
    df_month = df_month.loc[df_month['count_days'] >= min_days_month]
    df_mmf = (df_month.groupby(['Month'])
           .agg({'Water Year':'count', 'monthly_mean_flow':'mean'})
           .reset_index()
           .rename(columns={'Water Year':'count_months','monthly_mean_flow':'mean_monthly_flow'})
          )
    # Calculate mean annual, screening out years with too much missing data
    df_wy = (df.groupby(['Water Year'])
           .agg({'Day':'count', 'flow':'mean'})
           .reset_index()
           .rename(columns={'Day':'count_days','flow':'annual_mean_flow'})
          )
    df_wy = df_wy.loc[df_wy['count_days'] >= min_days_year]
    maf = df_wy['annual_mean_flow'].mean()
    # Calculate Tessman monthly bypass flows
    df_mmf.loc[df_mmf['mean_monthly_flow'] < 0.4 * maf, 'tessman_mbf'] = df_mmf['mean_monthly_flow']
    df_mmf.loc[0.4 * df_mmf['mean_monthly_flow'] >= 0.4 * maf, 'tessman_mbf'] = 0.4 * df_mmf['mean_monthly_flow']
    df_mmf.loc[(df_mmf['mean_monthly_flow'] >= 0.4 * maf) & (0.4 * df_mmf['mean_monthly_flow'] < 0.4 * maf), 'tessman_mbf'] = 0.4 * maf
    # Calculate the hydrograph assuming full diversion
    df = pd.merge(df,df_mmf, on="Month", how="left")
    max_div_allowed = df.flow.max() # placeholder until we develop a loi specific max diversion amount
    df['flow above mbf'] = np.where(df['flow'] > df['tessman_mbf'], df['flow'] - df['tessman_mbf'], 0)
    #df.loc[df['flow'] > df['tessman_mbf'], 'flow above mbf'] = df['flow'] - df['tessman_mbf']
    df['max diverted'] = np.where(df['flow above mbf']>max_div_allowed, max_div_allowed, df['flow above mbf'])
    df['Tessman flow req'] = df['flow'] - df['max diverted']
    df = df.set_index('datetime')
    df.index = pd.DatetimeIndex(df.index)
    dailyreqts.loc[:,l] = df.loc[dailyreqts.index,'Tessman flow req']

Indian


KeyError: "[Timestamp('2021-03-10 00:00:00'), Timestamp('2021-07-21 00:00:00'), Timestamp('2021-07-22 00:00:00'), Timestamp('2021-07-23 00:00:00')] not in index"

In [11]:

dailyreqts

,Indian,Sproul
1950-10-01,4.254344,NaN
1950-10-02,4.254344,NaN
1950-10-03,4.254344,NaN
1950-10-04,4.254344,NaN
1950-10-05,4.254344,NaN
1950-10-06,4.254344,NaN
1950-10-07,4.254344,NaN
1950-10-08,4.254344,NaN
1950-10-09,4.254344,NaN
1950-10-10,4.254344,NaN


In [12]:
dailyreqts = add_ts_col(dailyreqts)
dailyreqts.to_csv(out_csv)